In [79]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWLITEARM'
%run "../../Setup_Scripts/Setup_Generic.ipynb"

INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.
INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.
INFO: Found ChipWhisperer😍


In [80]:
scope.clock.clkgen_freq=100E6

In [81]:
scope.glitch.clk_src="clkgen"

In [82]:
scope.glitch.resetDCMs()

In [83]:
scope.glitch.output="enable_only"

In [84]:
#Reset chip - this is NEEDED for device to start.
#Also you can reset it 


import os

check_died = 0

reset_num = 0

def rest_power():
    global check_died,reset_num
    # 方法一：使用智能插座开关
    os.system(r"C:\platform-tools\adb.exe shell input tap 900 800")
    time.sleep(1)
    os.system(r"C:\platform-tools\adb.exe shell input tap 900 800")

    #等待系统完全启动
    time.sleep(60)
    
    check_died = 0

    reset_num = reset_num + 1
    print(reset_num)
    # 方法二：使用继电器开关
    # scope.io.tio3 = False
    # time.sleep(0.05)
    # scope.io.tio3 = True
    


In [85]:
def rest_login():
    target.flush()
    target.baud = 115200
    target.write("\n")
    target.write("pi\n")
    time.sleep(0.8)
    target.write("123456\n")
    time.sleep(0.8)
    target.write("./test\n")
    

In [86]:
def check_success():
    global check_died 
    val = target.simpleserial_read_witherrors('R', 25, end = '\n', glitch_timeout=50)#For loop check

    valid = val['valid']
    # if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']


    if "Result2" in raw_serial:
        if "-5894076284012091139" not in raw_serial:
            print("success")
            print(raw_serial)
            check_died = 0
            
            return True
             
        else:
            check_died = 0
            print("Glitching...")
            print(raw_serial)
            return False
     
    if "Result2" not in raw_serial:
            check_died = check_died + 1
            if check_died > 30:
                print(check_died)
                print("resert system")
                rest_power()
                rest_login()
                
    target.flush()
    

In [87]:
scope.io.glitch_hp=True
scope.glitch.width = 20
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])

gc.set_range("width", -49.8, 49.8)
gc.set_range("offset", 5, 49)
gc.set_global_step([5.0, 2.5])
gc.set_step("tries", 1)
gc.set_step("ext_offset", 1)

In [ ]:
import time 

rest_power()
rest_login()

time.sleep(1)

while True:
    for glitch_setting in gc.glitch_values():

        scope.glitch.offset = glitch_setting[1]
        scope.glitch.width = glitch_setting[0]
        scope.glitch.ext_offset = glitch_setting[2]

        scope.glitch.repeat = 1190
        scope.glitch.manual_trigger()
        time.sleep(0.3)  # 暂停1秒
        
    
        val =  check_success()
        
        if val == True:
            print(glitch_setting[1])
            print(glitch_setting[0])
            print(glitch_setting[2])
            break
        else:
            continue
        
print("Done glitching")


1
Glitching...
Result1: -5894076284012091139, Result2: 1

Glitching...
Result1: -5894076284012091139, Result2: 2

Glitching...
Result1: -5894076284012091139, Result2: 3

Glitching...
Result1: -5894076284012091139, Result2: 4

Glitching...
Result1: -5894076284012091139, Result2: 5

Glitching...
Result1: -5894076284012091139, Result2: 6

Glitching...
Result1: -5894076284012091139, Result2: 7

Glitching...
Result1: -5894076284012091139, Result2: 8

Glitching...
Result1: -5894076284012091139, Result2: 9

Glitching...
Result1: -5894076284012091139, Result2: 10

Glitching...
Result1: -5894076284012091139, Result2: 11

Glitching...
Result1: -5894076284012091139, Result2: 12

Glitching...
Result1: -5894076284012091139, Result2: 13

Glitching...
Result1: -5894076284012091139, Result2: 14

Glitching...
Result1: -5894076284012091139, Result2: 15

Glitching...
Result1: -5894076284012091139, Result2: 16

Glitching...
Result1: -5894076284012091139, Result2: 17

Glitching...
Result1: -589407628401209

In [275]:
#测试代码

# import time 
# while True:
#     scope.glitch.repeat = 980
#     scope.glitch.manual_trigger()
#     time.sleep(0.35)  # 暂停1秒

KeyboardInterrupt: 